# Lesson 4: Using Pydantic Models for Structured LLM Output
- Use Pydantic models directly in your API calls to LLMs
- Reliably receive a properly structured response using a variety of different frameworks and LLM providers.

---

### Import all required libraries and set up your environment

In [1]:
# Import packages
from pydantic import BaseModel, Field, EmailStr
from typing import List, Literal, Optional
from openai import OpenAI
import instructor
import anthropic
from datetime import date

import os
from dotenv import load_dotenv
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

d:\Softwares\Anaconda3\envs\llm_env\Lib\site-packages\pydantic\plugin\_schema_validator.py:39: UserWarning: ModuleNotFoundError while loading the `logfire-plugin` Pydantic plugin, this plugin will not be installed.

ModuleNotFoundError("No module named 'rich'")
  plugins = get_plugins()


### Define The Pydantic Data Models

In [2]:
# Define the UserInput model for customer support queries
class UserInput(BaseModel):
    name: str
    email: EmailStr
    query: str
    order_id: Optional[int] = Field(
        None,
        description="5-digit order number (cannot start with 0)",
        ge=10000,
        le=99999
    )
    purchase_date: Optional[date] = None

# Define the CustomerQuery model that inherits from UserInput
class CustomerQuery(UserInput):
    priority: str = Field(
        ..., description="Priority level: low, medium, high"
    )
    category: Literal[
        'refund_request', 'information_request', 'other'
    ] = Field(..., description="Query category")
    is_complaint: bool = Field(
        ..., description="Whether this is a complaint"
    )
    tags: List[str] = Field(..., description="Relevant keyword tags")

#### Provide sample input and validate it using your model

In [3]:
# Define your input data as a JSON string
user_input_json = '''{
    "name": "Joe User",
    "email": "joe.user@example.com",
    "query": "I ordered a new computer monitor and it arrived with the screen cracked. This is the second time this has happened. I need a replacement ASAP.",
    "order_number": 12345,
    "purchase_date": "2025-12-31"
}'''

In [4]:
# Validate the user_input_json by creating a UserInput instance
user_input = UserInput.model_validate_json(user_input_json)

### Build a prompt and call the Anthropic API with the instructor package for structured output

In [5]:
prompt = (
    f"Analyze the following customer query {user_input} "
    f"and provide a structured response."
)

#### Load and call LLM with Instructor package

In [6]:
# Create the client with instructor
anthropic_client = instructor.from_anthropic(anthropic.Anthropic())

In [7]:
# Get the LLM response - Instructor Client performs any retries if required
response = anthropic_client.messages.create(
    model = "claude-3-7-sonnet-latest",
    max_tokens = 1024,
    messages=[

        {
            "role": "user",
            "content": prompt
        }
    ],
    response_model=CustomerQuery
)

In [8]:
# Inspect the returned structured data
print(type(response))
print(response.model_dump_json(indent=2))

<class '__main__.CustomerQuery'>
{
  "name": "Joe User",
  "email": "joe.user@example.com",
  "query": "I ordered a new computer monitor and it arrived with the screen cracked. This is the second time this has happened. I need a replacement ASAP.",
  "order_id": null,
  "purchase_date": "2025-12-31",
  "priority": "high",
  "category": "refund_request",
  "is_complaint": true,
  "tags": [
    "damaged item",
    "computer monitor",
    "replacement",
    "second occurrence"
  ]
}


### Use OpenAI's structured output API with your Pydantic schema

In [9]:
# Initialize OpenAI client and call passing CustomerQuery in your API call
openai_client = OpenAI()
response = openai_client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[{"role": "user", "content": prompt}],
    response_format=CustomerQuery
)
response_content = response.choices[0].message.content
print(type(response_content))
print(response_content)

<class 'str'>
{"name":"Joe User","email":"joe.user@example.com","query":"I ordered a new computer monitor and it arrived with the screen cracked. This is the second time this has happened. I need a replacement ASAP.","order_id":null,"purchase_date":"2025-12-31","priority":"high","category":"refund_request","is_complaint":true,"tags":["cracked_screen","replacement","urgent","repeat_issue"]}


### Additional advanced usage and inspection

In [10]:
# Validate the repsonse you got from the LLM
valid_data = CustomerQuery.model_validate_json(
    response_content
)
print(type(valid_data))
print(valid_data.model_dump_json(indent=2))

<class '__main__.CustomerQuery'>
{
  "name": "Joe User",
  "email": "joe.user@example.com",
  "query": "I ordered a new computer monitor and it arrived with the screen cracked. This is the second time this has happened. I need a replacement ASAP.",
  "order_id": null,
  "purchase_date": "2025-12-31",
  "priority": "high",
  "category": "refund_request",
  "is_complaint": true,
  "tags": [
    "cracked_screen",
    "replacement",
    "urgent",
    "repeat_issue"
  ]
}


### Try the responses API from OpenAI

In [11]:
response = openai_client.responses.parse(
    model="gpt-4o",
    input=[{"role": "user", "content": prompt}],
    text_format=CustomerQuery
)

print(type(response))

<class 'openai.types.responses.parsed_response.ParsedResponse[CustomerQuery]'>


In [12]:
# Investigate class inheritance structure of the OpenAI response
def print_class_inheritence(llm_response):
    for cls in type(llm_response).mro():
        print(f"{cls.__module__}.{cls.__name__}")

print_class_inheritence(response)

openai.types.responses.parsed_response.ParsedResponse[CustomerQuery]
openai.types.responses.parsed_response.ParsedResponse
openai.types.responses.response.Response
openai._models.GenericModel
openai._compat.GenericModel
openai.BaseModel
pydantic.main.BaseModel
typing.Generic
builtins.object


In [13]:
# Print the response type and content 
print(type(response.output_parsed))
print(response.output_parsed.model_dump_json(indent=2))

<class '__main__.CustomerQuery'>
{
  "name": "Joe User",
  "email": "joe.user@example.com",
  "query": "I ordered a new computer monitor and it arrived with the screen cracked. This is the second time this has happened. I need a replacement ASAP.",
  "order_id": null,
  "purchase_date": "2025-12-31",
  "priority": "high",
  "category": "refund_request",
  "is_complaint": true,
  "tags": [
    "damaged product",
    "replacement request",
    "urgent"
  ]
}


### Try out the Pydantic AI package for defining an agent and getting a structured response

In [ ]:
# Try out the Pydantic AI package for defining an agent and getting a structured response
from pydantic_ai import Agent
import nest_asyncio
nest_asyncio.apply()

agent = Agent(
    model="google-gla:gemini-2.0-flash",
    output_type=CustomerQuery,
)

response = agent.run_sync(prompt)